# Sesjon 2 – RAG-evaluering med ragas

Evaluer minimal RAG-pipeline ved hjelp av ragas-metrikker: svarrelevans, troverdighet, kontekstpresisjon.


# Scenario
Dette scenariet evaluerer en minimal Retrieval Augmented Generation (RAG)-pipeline lokalt. Vi:
- Definerer et lite syntetisk dokumentkorpus.
- Embeder dokumenter og implementerer en enkel likhetsbasert søker.
- Genererer faktabaserte svar ved hjelp av en lokal modell (Foundry Local / OpenAI-kompatibel).
- Beregner ragas-metrikker (`answer_relevancy`, `faithfulness`, `context_precision`).
- Støtter en RASK modus (env `RAG_FAST=1`) for kun å beregne svarrelevans for rask iterasjon.

Bruk denne notatboken for å validere at din lokale modell + embeddings-stabel produserer faktabaserte svar før du skalerer til større korpus.


### Forklaring: Installasjon av avhengigheter
Installerer nødvendige biblioteker:
- `foundry-local-sdk` for lokal modellhåndtering.
- `openai` klientgrensesnitt.
- `sentence-transformers` for tette embeddings.
- `ragas` + `datasets` for evaluering og beregning av metrikker.
- `langchain-openai` adapter for ragas LLM-grensesnitt.

Trygt å kjøre på nytt; hopp over hvis miljøet allerede er klargjort.


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### Forklaring: Kjerneimporter og Metrikker
Laster inn kjernebiblioteker og ragas-metrikker. Viktige elementer:
- SentenceTransformer for embeddings.
- `evaluate` + utvalgte ragas-metrikker.
- `Dataset` for å konstruere evalueringskorpus.
Disse importene utløser ikke eksterne kall (bortsett fra mulig modellbufferlasting for embeddings).


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### Forklaring: Lekekorpus og QA-fasitsvar
Definerer et lite minnebasert korpus (`DOCS`), et sett med bruker-spørsmål og forventede fasitsvar. Disse muliggjør rask og deterministisk beregning av metrikker uten eksterne datahentinger. I virkelige scenarier ville du tatt utgangspunkt i produksjonsspørsmål og kuraterte svar.


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### Forklaring: Service Init, Embeddings & Safety Patch
Initialiserer Foundry Local manager, anvender en sikkerhetsoppdatering for skjemaavvik for `promptTemplate`, løser modell-ID, oppretter en OpenAI-kompatibel klient, og forhåndsberegner tette embeddings for dokumentkorpuset. Dette setter opp gjenbrukbar tilstand for henting + generering.


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Forklaring: Retriever-funksjon
Definerer en enkel vektorsimilaritets-retriever som bruker prikkprodukt over normaliserte embeddings. Returnerer de topp-k dokumentene (k=2 som standard). I produksjon bør denne byttes ut med ANN-indeks (FAISS, Chroma, Milvus) for skalerbarhet og lavere ventetid.


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### Forklaring: Genereringsfunksjon
`generate` konstruerer en begrenset prompt (systemet instruerer om å bruke KUN konteksten) og kaller den lokale modellen. Lav temperatur (0.1) favoriserer nøyaktig utvinning fremfor kreativitet. Returnerer trimmet svartekst.


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### Forklaring: Initialisering av reserveklient
Sikrer at `client` eksisterer selv om en tidligere initialiseringscelle ble hoppet over eller mislyktes—forhindrer NameError under senere evalueringssteg.


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### Forklaring: Evalueringssløyfe og Metrikker
Bygger evalueringsdatasettet (påkrevede kolonner: spørsmål, svar, kontekster, fasitsvar, referanse) og itererer deretter gjennom utvalgte ragas-metrikker.

Optimalisering:
- FAST_MODE begrenser seg til svarrelevans for raske tester.
- Per-metrikk-sløyfe unngår full rekalkulering når én metrikk feiler.

Returnerer en ordbok med metrikk -> poengsum (NaN ved feil).


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
